In [1]:
import pandas as pd

colunas = ['PROCESSO', 'ROTULO_MANUAL', 'LEI']

df = pd.read_csv('../data/leis.csv', header=0, sep=',', quotechar='"', usecols=colunas)

In [2]:
 df.head()

,PROCESSO,ROTULO_MANUAL,LEI
0,00039604720138240039,CDC,9656/1998
1,00039604720138240039,CDC,616/2012
2,00039604720138240039,CDC,9656/1998
3,00039604720138240039,CDC,616/2012
4,00039604720138240039,CDC,9656/1998


Utitiza apenas label que contenham mais de 200 exemplos.

In [3]:
rotulos_ignorados = ['PRE', 'TIT', 'COM', 'DMI', 'FAM', 'OIE', 'AUT', 'SOC', 'SUC', 'RESCO', 'MAR', 'CIV', 'POS', 'INF', 'REG']

df.loc[df['ROTULO_MANUAL'].isin( rotulos_ignorados), 'ROTULO_MANUAL'] = 'NONE'

In [4]:
groupby_rotulo = df.groupby('ROTULO_MANUAL')

groupby_rotulo[['LEI']].count().sort_values(['LEI'], ascending=False)

,LEI
ROTULO_MANUAL,
EXP,905
SEG,803
OIG,694
BAN,654
NONE,546
BUS,334
FAL,261
CON,257
SFH,210


In [5]:
df.shape

(5197, 3)

In [6]:
data = df.groupby(['PROCESSO', 'ROTULO_MANUAL', 'LEI']).size().unstack('LEI', fill_value=0)
data = data.reset_index()

data.head()

LEI,PROCESSO,ROTULO_MANUAL,1/1975,1/2012,100/2015,101/2010,10144/2002,10150/2000,1025/2009,10257/2001,...,9610/1998,9611/1998,9612/1998,9615/1998,9656/1988,9656/1997,9656/1998,9658/1998,9800/1999,9870/1999
0,00000042920148240058,CON,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,00000057320168240242,NONE,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,00000110319958240053,NONE,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,00000186020108240023,NONE,0,0,0,0,0,0,0,0,...,4,0,0,0,0,0,0,0,0,0
4,00000224520138240071,OIG,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
y = data['ROTULO_MANUAL'].values
y.shape

(1218,)

In [8]:
df1 = data.iloc[:,2:].copy()
df1.head()

LEI,1/1975,1/2012,100/2015,101/2010,10144/2002,10150/2000,1025/2009,10257/2001,104/1990,1060/1950,...,9610/1998,9611/1998,9612/1998,9615/1998,9656/1988,9656/1997,9656/1998,9658/1998,9800/1999,9870/1999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,4,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
x = df1.values
x.shape

(1218, 255)

In [10]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y)

import collections

print(collections.Counter(y_train))
print(collections.Counter(y_test))

Counter({'SEG': 153, 'OIG': 153, 'BAN': 122, 'NONE': 118, 'EXP': 77, 'BUS': 60, 'CON': 52, 'CDC': 36, 'RAI': 36, 'SFH': 35, 'DAN': 28, 'DPV': 23, 'FAL': 20})
Counter({'SEG': 47, 'OIG': 45, 'BAN': 42, 'NONE': 35, 'EXP': 31, 'CON': 21, 'BUS': 18, 'CDC': 16, 'SFH': 14, 'DPV': 12, 'RAI': 12, 'FAL': 7, 'DAN': 5})


In [11]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()

text_clf = Pipeline([
    #('enc', enc),
    ('clf', clf)
])

text_clf.fit(x , y)

Pipeline(memory=None,
     steps=[('clf', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'))])

Testando o modelo.

In [12]:
y_pred = text_clf.predict(x_test)

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)

accuracy

0.8950819672131147

In [13]:
from sklearn import metrics

print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         BAN       0.88      0.90      0.89        42
         BUS       0.85      0.94      0.89        18
         CDC       0.84      1.00      0.91        16
         CON       1.00      0.71      0.83        21
         DAN       0.62      1.00      0.77         5
         DPV       0.92      1.00      0.96        12
         EXP       0.97      0.97      0.97        31
         FAL       0.75      0.86      0.80         7
        NONE       0.90      0.74      0.81        35
         OIG       0.90      0.98      0.94        45
         RAI       0.73      0.67      0.70        12
         SEG       0.98      0.94      0.96        47
         SFH       0.86      0.86      0.86        14

   micro avg       0.90      0.90      0.90       305
   macro avg       0.86      0.89      0.87       305
weighted avg       0.90      0.90      0.89       305



In [14]:
from sklearn.metrics import confusion_matrix

cnf_matrix = confusion_matrix(y_test, y_pred)

In [15]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

      
df_cm = pd.DataFrame(cnf_matrix, range(13), range(13))
plt.figure(figsize = (10,7))
sn.set(font_scale=1.4)
sn.heatmap(df_cm, annot=True,annot_kws={"size": 16})

In [16]:
features = list(data.columns.values)
del features[0:2]

len(features)

255

In [17]:
df_sample = pd.DataFrame(columns=features)
df_sample = df_sample.append({'9656/1998': 1}, ignore_index=True)
df_sample = df_sample.fillna(0)

df_sample

,1/1975,1/2012,100/2015,101/2010,10144/2002,10150/2000,1025/2009,10257/2001,104/1990,1060/1950,...,9610/1998,9611/1998,9612/1998,9615/1998,9656/1988,9656/1997,9656/1998,9658/1998,9800/1999,9870/1999
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [18]:
y_pred = text_clf.predict(df_sample)

y_pred

array(['CDC'], dtype=object)

In [19]:
from sklearn.externals import joblib

joblib.dump(features, '../modelos/Features-Leis.pkl', compress = 1)
joblib.dump(clf, '../modelos/Modelo-Leis.pkl', compress = 1) 

['../modelos/Modelo-Leis.pkl']